In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
df = pd.read_csv("./new_data/data.csv",low_memory=False)
df = df.drop('test_date',axis=1)

df.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,0,0,0,0,0,negative,No,male,Other
1,0,1,0,0,0,negative,No,male,Other
2,0,0,0,0,0,negative,Yes,female,Other
3,0,0,0,0,0,negative,No,male,Other
4,0,1,0,0,0,negative,No,male,Contact with confirmed


In [4]:
df['corona_result'].replace({'negative':0,'other':0 , 'positive':1},inplace =True)
df['age_60_and_above'].replace({'No':0,np.nan:0,'Yes':1},inplace =True)
df['test_indication'].replace({'Other':0,'Abroad':0,'Contact with confirmed':1},inplace =True)
df['gender'].replace({np.nan:0,'female':0,'male':1},inplace =True)

In [5]:
print(df['test_indication'].unique())
print(df['age_60_and_above'].unique())
print(df['test_indication'].unique())
print(df['gender'].unique())

[0 1]
[0 1]
[0 1]
[1 0]


In [6]:
X=  pd.concat([ df.iloc[:,:5], df.iloc[:,6:7]], axis=1)
y = df.corona_result

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

In [9]:
regressor.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
cols = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache','gender']
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=cols)

,cough,fever,sore_throat,shortness_of_breath,head_ache,gender
0,0.057477,0.237054,0.110013,0.027411,0.556574,0.011471


In [11]:
from joblib import dump, load
dump(regressor, 'XGBRegressor.joblib')
#clf = load('model.joblib')

['XGBRegressor.joblib']

In [12]:
y_pred = regressor.predict(X_test)

In [13]:
y_pred

array([0.04858993, 0.04858993, 0.04858993, ..., 0.04858993, 0.04858993,
       0.05191216], dtype=float32)

In [14]:
mean_squared_error(y_test, y_pred)



0.0586591685408147

In [20]:
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.11%


In [16]:
xgb = load('XGBRegressor.joblib')

In [17]:
# exemple de test 
df1 = pd.DataFrame(np.array([[1, 1, 1,0, 0, 0]]),
                   columns=['cough' ,'fever' ,'sore_throat',
                            'shortness_of_breath','head_ache',
                            'age_60_and_above']
                  )
df2 = pd.DataFrame(np.array([[0, 0, 0,0, 0, 1]]),
                   columns=['cough' ,'fever' ,'sore_throat',
                            'shortness_of_breath','head_ache',
                            'age_60_and_above']
                  )

In [18]:
xgb = load('XGBRegressor.joblib')
xgb.predict(df1[0:])

array([0.62262565], dtype=float32)

In [19]:
def predictionXGB(df):
    xgb = load('XGBRegressor.joblib')
    prediction = xgb.predict(df[0:])
    if prediction[0] == 1:
        print("Vous etes positif au covid 19 ")
    else:
        print("Vous etes négatif au covid 19 ")

predictionXGB(df1)
predictionXGB(df2)

Vous etes négatif au covid 19 
Vous etes négatif au covid 19 
